In [1]:
from platform import python_version

print(python_version())

# %pip install langchain
# %pip install -U langchain-community
# %pip install pypdf
# %pip install spacy
# !python3.9 -m pip install spacy-llm
# !python3.9 -m spacy download en_core_web_sm
# %pip install scikit-learn
# %pip install -U sentence-transformers
# %pip install keybert
# %pip install transformers sentence-transformers nltk


3.9.21


In [2]:
import langchain
from langchain.document_loaders import PyPDFLoader
from langchain.docstore.document import Document
import os

# Directory containing your PDF files
directory_path = 'TCs'

# Initialize PyPDFLoader for each PDF in the directory
loaders = [PyPDFLoader(os.path.join(directory_path, f)) for f in os.listdir(directory_path) if f.endswith('.pdf')]

# Load documents from PDFs
tc_docs = []
for loader in loaders:
    tc_docs.extend(loader.load())

# Prepare the content and metadata for each news article as Document objects
tc_data = [
    Document(
        page_content=doc.page_content,  # Assuming this is how you access the page content of the document
        metadata={
            "source": doc.metadata['source'].removeprefix('TCs'),  # Assuming this is the metadata format
        }
    )
    for doc in tc_docs  # Assuming news_docs is a list of objects with page_content and metadata
]

data = tc_data[0].page_content
# print(type(news_articles_data))

page_contents = [doc.page_content for doc in tc_docs]
combined = "".join(page_contents) 
type(combined)

str

In [39]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=60
)
# split_docs = text_splitter.split_text(data)
split_docs = text_splitter.split_text(combined)
print(len(split_docs))

# function for functionality of pipeline
def split_text(combined_text):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=60
    )
    return text_splitter.split_text(combined_text)

150


### Entity & Relationships

In [42]:
import os
import json
import spacy
from collections import Counter
from pathlib import Path
from wasabi import msg
from spacy_llm.util import assemble

import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# traditional spacy NER
# def split_document_sent(text):
#     nlp = spacy.load("en_core_web_sm")
#     doc = nlp(text)
#     return [sent.text.strip() for sent in doc.sents] # referencial

def summarize_section(text):
    # Load SpaCy model
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)

    # Extract sentences
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]

    # Check if there are meaningful sentences
    if not sentences:
        return "No summary available"

    try:
        # Compute sentence embeddings using TF-IDF
        vectorizer = TfidfVectorizer(stop_words='english')
        sentence_vectors = vectorizer.fit_transform(sentences).toarray()

        # Calculate average vector and similarity
        avg_vector = np.mean(sentence_vectors, axis=0).reshape(1, -1)
        similarities = cosine_similarity(avg_vector, sentence_vectors)

        # Select the most relevant sentence
        most_relevant_index = np.argmax(similarities)
        return sentences[most_relevant_index]

    except ValueError:  # Handle cases with stop-words-only or empty input
        return "No summary available"

# spacy-llm relationship extraction
# def process_text(nlp, text, verbose=False):
#     doc = nlp(text)
#     if verbose:
#         msg.text(f"Text: {doc.text}")
#         msg.text(f"Entities: {[(ent.text, ent.label_) for ent in doc.ents]}")
#         msg.text("Relations:")
#         for r in doc._.rel:
#             msg.text(f"  - {doc.ents[r.dep]} [{r.relation}] {doc.ents[r.dest]}")
#     return doc

def run_pipeline(combined, config_path, examples_path=None, verbose=False):
    if not os.getenv("OPENAI_API_KEY"):
        msg.fail("OPENAI_API_KEY env variable was not found. Set it and try again.", exits=1)

    sections = split_text(combined)
    nlp = assemble(config_path, overrides={} if examples_path is None else {"paths.examples": str(examples_path)})

    # Initialize counters and storage
    processed_data = []
    entity_counts = Counter()
    relation_counts = Counter()

    for section in sections:
        summary = summarize_section(section)
        doc = nlp(summary)
        entities = [(ent.text, ent.label_) for ent in doc.ents]
        relations = [(doc.ents[r.dep].text, r.relation, doc.ents[r.dest].text) for r in doc._.rel]

        # Store processed data
        processed_data.append({
            'original_text': section,
            'summary': summary,
            'entities': entities,
            'relations': relations})

        # Update counters
        entity_counts.update([ent[1] for ent in entities])
        relation_counts.update([rel[1] for rel in relations])

    # Export to JSON
    with open('processed_data.json', 'w') as f:
        json.dump(processed_data, f)

    # Display summary
    msg.text(f"Entity counts: {entity_counts}")
    msg.text(f"Relation counts: {relation_counts}")

# Set your configuration paths and flags
config_path = Path("config.cfg")
examples_path = None  # or None if not using few-shot
verbose = True

# Run the pipeline
file = run_pipeline(combined, config_path, None, verbose)

/usr/local/lib/python3.9/site-packages/spacy_llm/pipeline/llm.py:143: UserWarning: Task supports sharding, but model does not provide context length. Data won't be sharded, prompt might exceed the model's context length. Set context length in your config. If you think spacy-llm should provide the context length for this model automatically, report this to https://github.com/explosion/spacy-llm/issues.
  warnings.warn(


Entity counts: Counter({'ORG': 263, 'GPE': 81, 'CARDINAL': 20, 'PERSON': 20,
'ORDINAL': 17, 'LAW': 13, 'WORK_OF_ART': 10, 'PRODUCT': 1, 'DATE': 1, 'EVENT':
1, 'TIME': 1})
Relation counts: Counter({'located_in': 44, 'related_to': 23, 'uses': 23,
'provides': 13, 'is_a': 6, 'includes': 6, 'supports': 6, 'refers_to': 6,
'affiliated_with': 5, 'has access to': 5, 'is a type of': 4, 'licensed_by': 3,
'runs': 3, 'is related to': 3, 'affects': 3, 'applies to': 3, 'included_with':
3, 'associated_with': 3, 'governs': 2, 'runs_on': 2, 'produces': 2, 'is
compatible with': 2, 'part_of': 2, 'does not endorse': 2, 'provides_service': 2,
'owns': 1, 'accompanied by': 1, 'obtained_from': 1, 'subject to': 1, 'set forth
in': 1, 'related to': 1, 'ownership': 1, 'produced_by': 1, 'supported_on': 1,
'branded_by': 1, 'downloads': 1, 'delivers': 1, 'facilitates': 1, 'authorizes':
1, 'stored_on': 1, 'governed_by': 1, 'has_rights_over': 1, 'license_for': 1,
'branded_as': 1, 'software_for': 1, 'prohibits': 1, 'bra

### Addition of description per chunk of text as well as simplification 

#### Add description

In [36]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def summarize_section(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]

    vectorizer = TfidfVectorizer(stop_words='english')
    # convert to array in order to perform cosine similarity
    sentence_vectors = vectorizer.fit_transform(sentences).toarray()

    avg_vector = np.mean(sentence_vectors, axis=0)
    avg_vector = avg_vector.reshape(1, -1)
    similarities = cosine_similarity(avg_vector, sentence_vectors)

    # index with highest cosine similarity to the average vector
    most_relevant_index = np.argmax(similarities)

    return sentences[most_relevant_index]


print(summarize_section(split_docs[2]))
split_docs[2]

To the extent that this software may be used to reproduce, modify, publish or 
distribute materials, it is licensed to you only for reproduction, modiﬁcation, publication and 
distribution of non-copyrighted materials, materials in which you own the copyright, or materials 
you are authorized or legally permitted to reproduce, modify, publish or distribute.


'IMPORTANT NOTE: To the extent that this software may be used to reproduce, modify, publish or \ndistribute materials, it is licensed to you only for reproduction, modiﬁcation, publication and \ndistribution of non-copyrighted materials, materials in which you own the copyright, or materials \nyou are authorized or legally permitted to reproduce, modify, publish or distribute. If you are \nuncertain about your right to copy, modify, publish or distribute any material, you should contact'

#### Simplification with Lexical Simplification with BERT

In [27]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
from sentence_transformers import SentenceTransformer, util
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np

nltk.download('punkt')

# Load models
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", trust_remote_code=True)
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased", trust_remote_code=True)
similarity_model = SentenceTransformer("all-MiniLM-L6-v2")

def mask_word(sentence, word):
    """Replace a word in the sentence with [MASK]."""
    return sentence.replace(word, "[MASK]")

def generate_candidates(masked_sentence, top_k=5):
    """Generate replacement candidates using MLM."""
    inputs = tokenizer(masked_sentence, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits

    mask_token_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
    mask_logits = logits[0, mask_token_index, :]
    top_tokens = mask_logits.topk(top_k, dim=-1).indices[0].tolist()

    candidates = [tokenizer.decode([token]).strip() for token in top_tokens]
    return candidates

def rank_candidates(sentence, word, candidates):
    """Rank candidates based on semantic similarity and simplicity."""
    original_embedding = similarity_model.encode(sentence)
    rankings = []

    for candidate in candidates:
        replaced_sentence = sentence.replace(word, candidate)
        candidate_embedding = similarity_model.encode(replaced_sentence)
        similarity = util.cos_sim(original_embedding, candidate_embedding)[0][0].item()
        rankings.append((candidate, similarity))

    # Sort by similarity and word length (shorter words preferred for simplicity)
    rankings.sort(key=lambda x: (-x[1], len(x[0])))
    return [candidate for candidate, _ in rankings]

def simplify_text(text, complexity_threshold=5):
    """Simplify text by replacing complex words."""
    simplified_sentences = []
    sentences = sent_tokenize(text)

    for sentence in sentences:
        words = word_tokenize(sentence)
        for word in words:
            if len(word) > complexity_threshold:  # Detect complex words by length
                masked_sentence = mask_word(sentence, word)
                candidates = generate_candidates(masked_sentence)
                ranked_candidates = rank_candidates(sentence, word, candidates)

                if ranked_candidates:
                    sentence = sentence.replace(word, ranked_candidates[0], 1)
        simplified_sentences.append(sentence)

    return " ".join(simplified_sentences)

# Example usage
text = """
The user shall be deemed to have consented to the terms and conditions herein referred to as the agreement. 
You agree to be bound by the obligations under this contract.
"""

simplified_text = simplify_text(text)
print("Simplified Text:", simplified_text)


[nltk_data] Downloading package punkt to /Users/meryjoy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Simplified Text: 
The user shall be deemed to have consent to the terms and conditions , referred to as the agreement. You agree to be bound by the contract under this contract.


Still work ahead for this part of the project, might be for later, not enough time

### Create Cypher Queries with eFLINT notation

In [46]:
def classify_and_generate_queries2(json_data, file_path):
    nodes = {}
    relationships = []

    # Enhanced mapping function for entity types based on eFLINT elements
    def map_eflint_type(entity_type, entity_name):
        # we are not really interested in numbers so we delete those
        # in the case of links it is better that we classify them as FACT rather than ORG
        if 'http://' in entity_type or 'https://' in entity_name:
            return 'FACT'

        if entity_type in ['CARDINAL', 'ORDINAL']:
            return 'NUMBER'


        if entity_type == 'ORG':
            if 'terms' in entity_name.lower() or 'conditions' in entity_name.lower() or 'agreement' in entity_name.lower():
                return 'DUTY'
            else:
                return 'ACTOR'

        if 'section' in entity_name.lower() or 'section' in entity_type.lower():
            return 'SECTION'
        
        mapping = {
            'PERSON': 'ACTOR',
            'EVENT': 'EVENT',
            'LAW': 'DUTY',
            'WORK_OF_ART': 'ACT',
            'CONDITION': 'CONDITION',
            'DATE': 'DATE',
            # they sadly do not appear in the NER we performed but we can nevertheless include them to keep the eFLINT semantics
            'CLAIMANT': 'CLAIMANT',
            'HOLDER': 'HOLDER'
        }
        return mapping.get(entity_type, 'FACT') 

    # entities and relationships
    for item in json_data:
        entities = item.get('entities', [])
        relations = item.get('relations', [])
        context_text = item.get('context', '')  # Text surrounding the entities

        # Generate section summary for the context
        section_summary = summarize_section(context_text) if context_text else "No summary available"
        
        for entity in entities:
            entity_name, entity_type = entity[:2]
            mapped_type = map_eflint_type(entity_type, entity_name)
            node_id = f"{entity_name.replace(' ', '_')}_{mapped_type}"
            nodes[node_id] = {
                'name': entity_name, 
                'type': mapped_type,
                'description': section_summary  # Add the summary as a description
            }
        
        for relation in relations:
            src_id = f"{relation[0].replace(' ', '_')}_{map_eflint_type(entities[0][1], entities[0][0])}"
            tgt_id = f"{relation[2].replace(' ', '_')}_{map_eflint_type(entities[0][1], entities[0][0])}"
            relationship_type = relation[1].replace(' ', '_').replace('-', '_')
            relationships.append((src_id, relationship_type, tgt_id))

    node_queries = [
        f"MERGE (n:{data['type']} {{name: '{data['name']}', description: '{data['description']}'}}) SET n.id = '{node_id}'"
        for node_id, data in nodes.items() if data['type'] != 'NUMBER'
    ]

    relationship_queries = [
        f"MATCH (a), (b) WHERE a.id = '{rel[0]}' AND b.id = '{rel[2]}' "
        f"MERGE (a)-[:{rel[1]}]->(b)"
        for rel in relationships
    ]

    queries = node_queries + relationship_queries
    # Save queries to a text file
    with open(file_path, 'w') as file:
        for query in queries:
            file.write(query + '\n')

    return queries

In [51]:
def classify_and_generate_queries(json_data, file_path):
    nodes = {}
    relationships = []

    # Enhanced mapping function for entity types based on eFLINT elements
    def map_eflint_type(entity_type, entity_name):
        if 'http://' in entity_type or 'https://' in entity_name:
            return 'FACT'
        if entity_type in ['CARDINAL', 'ORDINAL']:
            return 'NUMBER'
        if entity_type == 'ORG':
            if 'terms' in entity_name.lower() or 'conditions' in entity_name.lower() or 'agreement' in entity_name.lower():
                return 'DUTY'
            else:
                return 'ACTOR'
        if 'section' in entity_name.lower() or 'section' in entity_type.lower():
            return 'SECTION'
        mapping = {
            'PERSON': 'ACTOR',
            'EVENT': 'EVENT',
            'LAW': 'DUTY',
            'WORK_OF_ART': 'ACT',
            'CONDITION': 'CONDITION',
            'DATE': 'DATE',
        }
        return mapping.get(entity_type, 'FACT')

    # Process summaries and associate them with entities
    for item in json_data:
        summary = item.get('summary', 'No summary available')
        entities = item.get('entities', [])
        relations = item.get('relations', [])

        for entity in entities:
            entity_name, entity_type = entity[:2]
            mapped_type = map_eflint_type(entity_type, entity_name)
            node_id = f"{entity_name.replace(' ', '_')}_{mapped_type}"
            if node_id not in nodes:
                nodes[node_id] = {
                    'name': entity_name,
                    'type': mapped_type,
                    'descriptions': [summary],  # Start with the current summary
                }
            else:
                nodes[node_id]['descriptions'].append(summary)  # Append additional summaries

        # Add relationships if present
        for relation in relations:
            src_id = f"{relation[0].replace(' ', '_')}_{map_eflint_type(entity_type, relation[0])}"
            tgt_id = f"{relation[2].replace(' ', '_')}_{map_eflint_type(entity_type, relation[2])}"
            relationship_type = relation[1].replace(' ', '_').replace('-', '_')
            relationships.append((src_id, relationship_type, tgt_id))

    # Create Cypher queries for nodes
    node_queries = [
        f"""
        MERGE (n:{data['type']} {{name: '{data['name']}'}})
        SET n.id = '{node_id}', n.descriptions = {json.dumps(data['descriptions'])}
        """
        for node_id, data in nodes.items() if data['type'] != 'NUMBER'
    ]

    # Create Cypher queries for relationships
    relationship_queries = [
        f"""
        MATCH (a), (b)
        WHERE a.id = '{rel[0]}' AND b.id = '{rel[2]}'
        MERGE (a)-[:{rel[1]}]->(b)
        """
        for rel in relationships
    ]

    queries = node_queries + relationship_queries

    # Save queries to a text file
    with open(file_path, 'w') as file:
        for query in queries:
            file.write(query + '\n')

    return queries


In [52]:
with open('processed_data.json', 'r') as file:
    json_data = json.load(file)

queries = classify_and_generate_queries(json_data, 'cypher_queries.txt')

In [55]:
from neo4j import GraphDatabase

def execute_queries(queries, uri, user, password):
    driver = GraphDatabase.driver(uri, auth=(user, password))
    with driver.session() as session:
        for query in queries:
            session.run(query)
    driver.close()

# Example usage
uri = "neo4j://localhost:7687"
user = "neo4j"
password = "movies11"

execute_queries(queries, uri, user, password)


## Other option: 
### Using Langchain for Queries

In [ ]:
# %pip install langchain_openai
# %pip install langchain_experimental
# %pip install neo4j langchain-neo4j
# %pip install yfiles_jupyter_graphs

In [16]:
if not os.getenv("NEO4J_URI"):
    print("nope")
if not os.getenv("NEO4J_USERNAME"):
    print("nope")
if not os.getenv("NEO4J_PASSWORD"):
    print("nope")

In [17]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
tc_document = text_splitter.split_documents(tc_data)

In [18]:
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm=ChatOpenAI(temperature=0, model_name="gpt-4o-mini")
     
llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(tc_document)

In [19]:
from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph()
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [20]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 80"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [21]:
# showGraph()

## Summarization based RAG

In [22]:
from langchain_openai import OpenAIEmbeddings
from langchain_neo4j import Neo4jVector
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain.document_loaders import PyPDFLoader
from langchain.docstore.document import Document
from langchain.prompts import ChatPromptTemplate
from langchain_openai import OpenAI
from pydantic import BaseModel, Field
from typing import Tuple, List, Optional
import os
import json

In [23]:
# Establishing a connection to the Neo4j Graph
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [24]:
# Define the graph query for fulltext index if not already existing
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [25]:
# BaseModel for extracting entities
class Entities(BaseModel):
    """Identifying information about entities."""
    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Extracting key entities and information for summarization."),
        ("human", "Summarize the following content using key entities and relationships: {context}"),
    ]
)

llm = OpenAI() 

In [ ]:
# function to generate fulltext search query
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f"{word}~2 AND"
    full_text_query += f"{words[-1]}~2"
    return full_text_query.strip()

# very basic retriever
def structured_retriever(context: str) -> str:
    result = ""
    entities = entity_chain.invoke({"context": context})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
                WITH node
                MATCH (node)-[r:!MENTIONS]->(neighbor)
                RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
                UNION ALL
                WITH node
                MATCH (node)<-[r:!MENTIONS]-(neighbor)
                RETURN neighbor.id + ' - ' + type(r) + ' -> ' + node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result